___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Non-Negative Matric Factorization

Let's repeat thet opic modeling task from the previous lecture, but this time, we will use NMF instead of LDA.

## Data

We will be using articles scraped from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv("npr.csv")

In [3]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [5]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words="english")

In [6]:
dtm = tfidf.fit_transform(npr["Article"])

In [7]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## NMF

In [8]:
from sklearn.decomposition import NMF

In [9]:
nmf_model = NMF(n_components=7, random_state=42)

In [10]:
# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit(dtm)

NMF(n_components=7, random_state=42)

## Displaying Topics

In [11]:
len(tfidf.get_feature_names())

/home/saisreekar/anaconda3/envs/my_env/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


54777

In [12]:
import random

In [13]:
for i in range(10):
    random_word_id = random.randint(0, 54776)
    print(tfidf.get_feature_names()[random_word_id])

elusiveness
disempowered
zulu
60th
extremist
hoosiers
jessie
lipstick
survivors
commences


In [14]:
for i in range(10):
    random_word_id = random.randint(0, 54776)
    print(tfidf.get_feature_names()[random_word_id])

extracurriculars
rashid
flopped
710
exacts
zaire
motifs
breastplate
swaying
economic


In [15]:
len(nmf_model.components_)

7

In [16]:
nmf_model.components_

array([[0.00000000e+00, 2.51270507e-01, 0.00000000e+00, ...,
        1.71376078e-03, 2.39241571e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.23172575e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 3.12287576e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.90615701e-03, 0.00000000e+00, 1.50483058e-03, ...,
        7.06176596e-04, 5.86173445e-04, 6.90910657e-04],
       [4.01955802e-03, 5.31379095e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [17]:
len(nmf_model.components_[0])

54777

In [18]:
single_topic = nmf_model.components_[0]

In [19]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([    0, 27208, 27206, ..., 36283, 54692, 42993])

In [20]:
# Word least representative of this topic
single_topic[18302]

0.0

In [21]:
# Word most representative of this topic
single_topic[42993]

2.0162023252917707

In [22]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([14441, 36310, 53989, 52615, 47218, 53152, 19307, 36283, 54692,
       42993])

In [23]:
top_word_indices = single_topic.argsort()[-10:]

In [24]:
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

disease
percent
women
virus
study
water
food
people
zika
says


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [25]:
for index, topic in enumerate(nmf_model.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print("\n")

THE TOP 15 WORDS FOR TOPIC #0
['new', 'research', 'like', 'patients', 'health', 'disease', 'percent', 'women', 'virus', 'study', 'water', 'food', 'people', 'zika', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['gop', 'pence', 'presidential', 'russia', 'administration', 'election', 'republican', 'obama', 'white', 'house', 'donald', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['senate', 'house', 'people', 'act', 'law', 'tax', 'plan', 'republicans', 'affordable', 'obamacare', 'coverage', 'medicaid', 'insurance', 'care', 'health']


THE TOP 15 WORDS FOR TOPIC #3
['officers', 'syria', 'security', 'department', 'law', 'isis', 'russia', 'government', 'state', 'attack', 'president', 'reports', 'court', 'said', 'police']


THE TOP 15 WORDS FOR TOPIC #4
['primary', 'cruz', 'election', 'democrats', 'percent', 'party', 'delegates', 'vote', 'state', 'democratic', 'hillary', 'campaign', 'voters', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['love', 've', 'don', 'al

### Attaching Discovered Topic Labels to Original Articles

In [26]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [27]:
dtm.shape

(11992, 54777)

In [28]:
len(npr)

11992

In [29]:
topic_results = nmf_model.transform(dtm)

In [30]:
topic_results.shape

(11992, 7)

In [31]:
topic_results[0]

array([0.        , 0.12079653, 0.00139891, 0.05915242, 0.01519226,
       0.        , 0.        ])

In [32]:
topic_results[0].round(2)

array([0.  , 0.12, 0.  , 0.06, 0.02, 0.  , 0.  ])

In [33]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [34]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [35]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 0, 4, 3])

In [36]:
npr["Topic"] = topic_results.argmax(axis=1)

In [37]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",6
5,I did not want to join yoga class. I hated tho...,5
6,With a who has publicly supported the debunk...,0
7,"I was standing by the airport exit, debating w...",0
8,"If movies were trying to be more realistic, pe...",0
9,"Eighteen years ago, on New Year’s Eve, David F...",5


## Great work!